### 모델 학습 코드, 배포 모델은 question_model.py 확인

In [1]:
import pandas as pd
train = pd.read_csv('fci.csv')
train.head(5)

,0,1,label
0,2,현재 집안 기온이 어떻게 되니,1
1,2,동기 모임 장소가 어디야,1
2,2,문학과 예술 중에서 추천할 만한 강좌 있어,1
3,2,다음달 필리핀 최고 기온 좀 검색해줘,1
4,2,거실에 불이 켜져 있는지 보고 와,1


In [2]:
train['label'].value_counts()

1    17869
0    17869
Name: label, dtype: int64

In [3]:
from transformers import ElectraTokenizer

train_X = train['1'].tolist()

tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
train_X = tokenizer(train_X, truncation=True, padding=True)

In [4]:
import numpy as np
train_toks = np.asarray(train_X['input_ids'])
train_masks = np.asarray(train_X['attention_mask'])

train_y = np.asarray(train['label'])

print(train_toks.shape)
print(train_masks.shape)
print(train_y.shape)

emb_len = train_toks.shape[-1]

(35738, 62)
(35738, 62)
(35738,)


In [5]:
from transformers import logging, TFElectraModel
logging.set_verbosity_error()
import tensorflow as tf

electra_model = TFElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator', from_pt=True)

input_toks = tf.keras.layers.Input(shape=(emb_len,), name='toks', dtype='int32')
input_masks = tf.keras.layers.Input(shape=(emb_len,), name='masks', dtype='int32')

electra_output = electra_model(input_toks, attention_mask=input_masks).last_hidden_state

x = tf.keras.layers.GlobalAveragePooling1D()(electra_output)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
y = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=[input_toks, input_masks], outputs=y)
adam = tf.keras.optimizers.Adam(learning_rate=4e-5)

model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
toks (InputLayer)               [(None, 62)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 62)]         0                                            
__________________________________________________________________________________________________
tf_electra_model (TFElectraMode TFBaseModelOutput(la 112330752   toks[0][0]                       
                                                                 masks[0][0]                      
__________________________________________________________________________________________________
global_average_pooling1d (Globa (None, 768)          0           tf_electra_model[0][0]

In [7]:
history = model.fit(x={'toks':train_toks, 'masks':train_masks}, y=train_y, epochs=5, validation_split=0.2)

Epoch 1/5
894/894 [==============================] - 399s 447ms/step - loss: 0.0930 - accuracy: 0.9683 - val_loss: 0.1307 - val_accuracy: 0.9604
Epoch 2/5
894/894 [==============================] - 394s 441ms/step - loss: 0.0468 - accuracy: 0.9843 - val_loss: 0.1111 - val_accuracy: 0.9629
Epoch 3/5
894/894 [==============================] - 403s 451ms/step - loss: 0.0308 - accuracy: 0.9896 - val_loss: 0.1337 - val_accuracy: 0.9605
Epoch 4/5
 22/894 [..............................] - ETA: 5:39 - loss: 0.0258 - accuracy: 0.9886

KeyboardInterrupt: 

In [6]:
model.fit(x={'toks':train_toks, 'masks':train_masks}, y=train_y, epochs=2)

Epoch 1/2
1117/1117 [==============================] - 464s 415ms/step - loss: 0.0919 - accuracy: 0.9667
Epoch 2/2
1117/1117 [==============================] - 475s 425ms/step - loss: 0.0451 - accuracy: 0.9846


In [12]:
test_X = tokenizer(['18학번 요람 보니까 졸업까지 18학점 이수하라고 되어 있던데 이거 맞아요'], truncation=True, padding='max_length', max_length=emb_len)
test_toks = np.asarray(test_X['input_ids'])
test_masks = np.asarray(test_X['attention_mask'])

print(test_toks.shape)
print(test_masks.shape)

(1, 62)
(1, 62)


In [13]:
model.predict(x={'toks':test_toks, 'masks':test_masks})

array([[0.14322628, 0.85677373]], dtype=float32)

In [14]:
model.save_weights('question_weights/')

In [6]:
model.load_weights('question_weights/')

In [8]:
test_texts = ['어대역에서 새천년관까지 어떻게 가요', 
                '신공학관 1층에도 있고 학생회관 2층에도 있음',
                '행정관 앞에 있는 치즈가 만쥬가 맞나요',
                '제발 이상한 소리 좀 하지마',
                '우리 등록금 언제까지 납부임']

In [9]:
test_X = tokenizer(test_texts, truncation=True, padding='max_length', max_length=emb_len)
test_toks = np.asarray(test_X['input_ids'])
test_masks = np.asarray(test_X['attention_mask'])

pred = model.predict(x={'toks':test_toks, 'masks':test_masks})

#print(pred)

for i, entry in enumerate(test_texts):
    if pred[i][0] >= 0.5:
        print(entry, '-> 질문이 아닙니다')
    else:
        print(entry, '-> 질문입니다')

어대역에서 새천년관까지 어떻게 가요 -> 질문입니다
신공학관 1층에도 있고 학생회관 2층에도 있음 -> 질문이 아닙니다
행정관 앞에 있는 치즈가 만쥬가 맞나요 -> 질문입니다
제발 이상한 소리 좀 하지마 -> 질문이 아닙니다
우리 등록금 언제까지 납부임 -> 질문입니다
